# スタッキング003

In [1]:
import torch
# GPUの名称、CUDA Compute Capabilityを取得
print(torch.cuda.get_device_name())

NVIDIA GeForce RTX 2080 Ti


In [2]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import pickle
import optuna
import re
import gc

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline
from FeatureEngineering import FeatureEngineering
from FeatureEngineering2 import FeatureEngineering2
from FeatureEngineering3 import FeatureEngineering3

from models import Model1Xgb, Model1Lgb, Model1NN, Model2Linear

%matplotlib inline
gc.enable()

print("OK\n")

OK



## 共通関数定義

In [3]:
### LightGBM
params_lgb = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}
### Xgboost
params_xgb = {
        'booster': 'gbtree'
        ,'objective': 'binary:logistic'
        ,'silent': 1
        ,'random_state': 17
        ,'eval_metric': 'auc'
        ,'eta': 0.1
        ,'max_depth': 8
        ,'subsample': 0.9
        ,'colsample_state': 71
        ,'colsample_bytree': 1.0
        ,'colsample_bylevel': 0.3
        ,'gamma': 0
        ,'lambda': 1
        ,'alpha': 0
        ,'min_child_weight': 1
        ,'gpu_id': 0
        ,'tree_method': 'gpu_hist'
}

In [4]:
# 学習データに対する「目的変数を知らない」予測値と、テストデータに対する予測値を返す関数
def predict_cv(model, train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_idxes = []

    kf = KFold(n_splits=4, shuffle=True, random_state=71)

    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y, va_x, va_y)
        pred = model.predict(va_x)
        print(pred)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)

    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)

    return pred_train, preds_test

In [5]:
params_base = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}



def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "num_leaves": trial.suggest_int("num_leaves", 8, 256)
            ,"min_child_samples": trial.suggest_int("min_child_samples", 5, 200)
            ,"min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True)
            ,"feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0)
            ,"bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0)
            ,"lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e+2, log=True)
            ,"lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e+2, log=True)
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = lgb.LGBMClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
                    ,early_stopping_rounds=100
                    ,verbose=0
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

## インスタンス生成

In [6]:
baseline = Baseline('SK_ID_CURR')
modules = Modules('SK_ID_CURR')
FeatureEngineering = FeatureEngineering('TARGET')
FeatureEngineering2 = FeatureEngineering2('TARGET')
FeatureEngineering3 = FeatureEngineering3('TARGET')

## 特徴量生成1

In [7]:
apps, prev, bureau, bureau_bal, pos_bal, install, card_bal = FeatureEngineering3.get_dataset()
apps.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0.0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0.0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0.0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0.0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df = FeatureEngineering3.get_apps_all_with_all_agg(apps, prev, bureau, bureau_bal, pos_bal, install, card_bal)
df.head()

prev_agg shape: (338857, 41) bureau_agg shape: (305811, 103)
pos_bal_agg shape: (337252, 45) install_agg shape: (339587, 59) card_bal_agg shape: (103558, 70)
apps_all before merge shape: (356255, 152)
apps_all after merge with all shape: (356255, 466)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CARD_M3BALANCE_LIMIT_RATIO_MIN,CARD_M3BALANCE_LIMIT_RATIO_MAX,CARD_M3DRAWING_LIMIT_RATIO_MIN,CARD_M3DRAWING_LIMIT_RATIO_MAX,CARD_M3CARD_IS_DPD_MEAN,CARD_M3CARD_IS_DPD_SUM,CARD_M3CARD_IS_DPD_UNDER_120_MEAN,CARD_M3CARD_IS_DPD_UNDER_120_SUM,CARD_M3CARD_IS_DPD_OVER_120_MEAN,CARD_M3CARD_IS_DPD_OVER_120_SUM
0,100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0.0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0.0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0.0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100007,0.0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
### one-hotencoding
encoding_columns = []

#### one-hotencoding処理対象列を取得
for col in df.columns:
    if df[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

NAME_CONTRACT_TYPE
CODE_GENDER
FLAG_OWN_CAR
FLAG_OWN_REALTY
NAME_TYPE_SUITE
NAME_INCOME_TYPE
NAME_EDUCATION_TYPE
NAME_FAMILY_STATUS
NAME_HOUSING_TYPE
OCCUPATION_TYPE
WEEKDAY_APPR_PROCESS_START
ORGANIZATION_TYPE
FONDKAPREMONT_MODE
HOUSETYPE_MODE
WALLSMATERIAL_MODE
EMERGENCYSTATE_MODE


In [10]:
# general cleaning procedures
df = df[df['CODE_GENDER'] != 'XNA']
df = df[df['AMT_INCOME_TOTAL'] < 20000000] # remove a outlier 117M
# NaN values for DAYS_EMPLOYED: 365.243 -> nan
df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True) # set null value
df['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True) # set null value

### CODE_GENDER, 
# Categorical features with Binary encode (0 or 1; two categories)
for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
    df[bin_feature], uniques = pd.factorize(df[bin_feature])

### NAME_INCOME_TYPE
df['NAME_INCOME_TYPE'].replace(['Maternity leave'], ['Unemployed'], inplace=True)
df['NAME_INCOME_TYPE'].replace(['Working','Commercial associate','Pensioner','State servant','Unemployed','Student','Businessman','Maternity leave'], [0, 1, 2, 3, 4, 5, 6, 7], inplace=True)
### NAME_FAMILY_STATUS
df['NAME_FAMILY_STATUS'].replace('Unknown', 'Married', inplace=True)
df['NAME_FAMILY_STATUS'].replace(['Married','Single / not married','Civil marriage','Separated','Widow'], [0, 1, 2, 3, 4], inplace=True)

### 
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x))

df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CARD_M3BALANCE_LIMIT_RATIO_MIN,CARD_M3BALANCE_LIMIT_RATIO_MAX,CARD_M3DRAWING_LIMIT_RATIO_MIN,CARD_M3DRAWING_LIMIT_RATIO_MAX,CARD_M3CARD_IS_DPD_MEAN,CARD_M3CARD_IS_DPD_SUM,CARD_M3CARD_IS_DPD_UNDER_120_MEAN,CARD_M3CARD_IS_DPD_UNDER_120_SUM,CARD_M3CARD_IS_DPD_OVER_120_MEAN,CARD_M3CARD_IS_DPD_OVER_120_SUM
0,100002,1.0,Cash loans,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0.0,Cash loans,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0.0,Revolving loans,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0.0,Cash loans,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100007,0.0,Cash loans,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
#### one-hotencoding実行
for col in encoding_columns:
    df = pd.get_dummies(df, columns=[col], dummy_na=True)

df.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_nan
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,0,0,0,0,1,0,0,1,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,0,0,0,0,0,0,0,1,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,0,0,0,0,0,0,1,0,0,1
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,0,0,0,0,0,0,1,0,0,1
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,0,0,0,0,0,0,1,0,0,1


In [12]:
### infをnanへ置換
df = df.replace([np.inf, -np.inf], np.nan)

In [13]:
### 変数選択001
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 603 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 593 features are remained after removing non-informative features
60 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 533 features are remained after removing features not interesting for LightGBM classifier
---=> 38 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 571 features

---=> df final shape: (356250, 571)  <=--- 



,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,POS_M20MONTHS_BALANCE_MAX_high_risk,POS_M20MONTHS_BALANCE_MAX_low_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,CARD_M3CARD_IS_DPD_OVER_120_SUM_high_risk,CARD_M3CARD_IS_DPD_OVER_120_SUM_low_risk
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,...,0,1,0,0,0,0,0,0,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,...,1,0,0,0,0,0,0,0,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,...,0,0,0,0,0,0,0,0,0,0
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,...,0,1,1,0,1,0,1,0,1,0
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,...,0,1,0,0,0,0,0,0,0,0


In [14]:
### 変数選択002
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 571 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 567 features are remained after removing non-informative features
22 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 545 features are remained after removing features not interesting for LightGBM classifier
---=> 15 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 560 features

---=> df final shape: (356250, 560)  <=--- 



,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,PREV_PREV_GOODS_DIFF_MAX_low_risk,BUREAU_BUREAU_IS_DPD_SUM_high_risk,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,...,0,0,1,1,0,0,0,0,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,...,0,0,1,1,0,0,0,0,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,...,0,0,1,0,0,0,0,0,0,0
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,...,0,0,0,0,1,0,1,0,1,0
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,...,0,0,1,0,0,0,0,0,0,0


In [15]:
# infをnanへ置換
df = df.replace([np.inf, -np.inf], np.nan)
df.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,PREV_PREV_GOODS_DIFF_MAX_low_risk,BUREAU_BUREAU_IS_DPD_SUM_high_risk,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,...,0,0,1,1,0,0,0,0,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,...,0,0,1,1,0,0,0,0,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,...,0,0,1,0,0,0,0,0,0,0
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,...,0,0,0,0,1,0,1,0,1,0
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,...,0,0,1,0,0,0,0,0,0,0


In [16]:
### 変数名から不正な文字を除去
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
print(df.shape)
df.head()

(356250, 560)


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,PREV_PREV_GOODS_DIFF_MAX_low_risk,BUREAU_BUREAU_IS_DPD_SUM_high_risk,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,...,0,0,1,1,0,0,0,0,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,...,0,0,1,1,0,0,0,0,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,...,0,0,1,0,0,0,0,0,0,0
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,...,0,0,0,0,1,0,1,0,1,0
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,...,0,0,1,0,0,0,0,0,0,0


In [17]:
df_train, df_test = FeatureEngineering3.get_apps_all_train_test(df)
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,PREV_PREV_GOODS_DIFF_MAX_low_risk,BUREAU_BUREAU_IS_DPD_SUM_high_risk,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,...,0,0,1,1,0,0,0,0,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,...,0,0,1,1,0,0,0,0,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,...,0,0,1,0,0,0,0,0,0,0
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,...,0,0,0,0,1,0,1,0,1,0
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,...,0,0,1,0,0,0,0,0,0,0


In [18]:
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,PREV_PREV_GOODS_DIFF_MAX_low_risk,BUREAU_BUREAU_IS_DPD_SUM_high_risk,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk
307511,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,0,0,1,1,0,0,0,0,0,0
307512,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,0,0,1,1,0,0,0,0,0,0
307513,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,0,0,1,0,1,0,1,0,1,0
307514,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,0,0,1,1,1,0,1,0,1,0
307515,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
pred_train_1b = pd.read_csv('./home-credit-default-risk/exports/pred_train_1b.csv')
pred_test_1b =  pd.read_csv('./home-credit-default-risk/exports/pred_test_1b.csv')
pred_test_1b.head()

,SK_ID_CURR,pred_1b
0,100001,0.051055
1,100005,0.166067
2,100013,0.043899
3,100028,0.056542
4,100038,0.183168


In [20]:
df_train = df_train.merge(pred_train_1b, how='left', on='SK_ID_CURR')
df_test = df_test.merge(pred_test_1b, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,BUREAU_BUREAU_IS_DPD_SUM_high_risk,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,0,1,1,0,0,0,0,0,0,0.051055
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,0,1,1,0,0,0,0,0,0,0.166067
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,0,1,0,1,0,1,0,1,0,0.043899
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,0,1,1,1,0,1,0,1,0,0.056542
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0,0,0,0,0,0,0,0.183168


In [21]:
pred_train_1a = pd.read_csv('./home-credit-default-risk/exports/pred_train_1a.csv')
pred_test_1a =  pd.read_csv('./home-credit-default-risk/exports/pred_test_1a.csv')
pred_test_1b.head()

,SK_ID_CURR,pred_1b
0,100001,0.051055
1,100005,0.166067
2,100013,0.043899
3,100028,0.056542
4,100038,0.183168


In [22]:
df_train = df_train.merge(pred_train_1a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(pred_test_1a, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,1,1,0,0,0,0,0,0,0.051055,0.038227
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,1,1,0,0,0,0,0,0,0.166067,0.112768
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,1,0,1,0,1,0,1,0,0.043899,0.033638
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,1,1,1,0,1,0,1,0,0.056542,0.031213
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0,0,0,0,0,0,0.183168,0.181109


In [23]:
train_set2a = pd.read_csv('./home-credit-default-risk/exports/train_set2a.csv')
test_set2a =  pd.read_csv('./home-credit-default-risk/exports/test_set2a.csv')
test_set2a.head()

,SK_ID_CURR,set2a
0,100001,0.025023
1,100005,0.090335
2,100013,0.037033
3,100028,0.036644
4,100038,0.172674


In [24]:
df_train = df_train.merge(train_set2a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set2a, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a,set2a
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,1,0,0,0,0,0,0,0.051055,0.038227,0.025023
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,1,0,0,0,0,0,0,0.166067,0.112768,0.090335
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,0,1,0,1,0,1,0,0.043899,0.033638,0.037033
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,1,1,0,1,0,1,0,0.056542,0.031213,0.036644
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0,0,0,0,0,0.183168,0.181109,0.172674


In [25]:
### infをnanへ置換
df_train = df_train.replace([np.inf, -np.inf], np.nan)
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a,set2a
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,...,1,0,0,0,0,0,0,0.516856,0.162368,0.156048
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,...,1,0,0,0,0,0,0,0.010645,0.027249,0.012034
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,...,0,0,0,0,0,0,0,0.054077,0.030869,0.026333
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,...,0,1,0,1,0,1,0,0.059492,0.029480,0.029856
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,...,0,0,0,0,0,0,0,0.076165,0.047569,0.034698


In [26]:
### infをnanへ置換
df_test = df_test.replace([np.inf, -np.inf], np.nan)
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a,set2a
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,1,0,0,0,0,0,0,0.051055,0.038227,0.025023
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,1,0,0,0,0,0,0,0.166067,0.112768,0.090335
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,0,1,0,1,0,1,0,0.043899,0.033638,0.037033
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,1,1,0,1,0,1,0,0.056542,0.031213,0.036644
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0,0,0,0,0,0.183168,0.181109,0.172674


In [27]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(df_test.select_dtypes(include='number').columns)
encoding_columns.remove('SK_ID_CURR')

In [28]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a,set2a
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,-3648.0,...,1,0,0,0,0,0,0,0.516856,0.162368,0.156048
100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,-1186.0,...,1,0,0,0,0,0,0,0.010645,0.027249,0.012034
100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,-4260.0,...,0,0,0,0,0,0,0,0.054077,0.030869,0.026333
100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,-9832.0,...,0,1,0,1,0,1,0,0.059492,0.029480,0.029856
100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,-4312.0,...,0,0,0,0,0,0,0,0.076165,0.047569,0.034698


In [29]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a,set2a
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1.0,-0.577531,0.313221,-0.478099,-0.166152,-0.507239,-0.149780,1.506891,0.747070,0.379883,...,0.645083,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,5.053162,0.986709,0.860543
100003,0.0,-0.577531,0.933556,1.725433,0.592656,1.599963,-1.252930,-0.166811,0.511719,1.079102,...,0.645083,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.839862,-0.548398,-0.659948
100004,0.0,-0.577531,-0.927449,-1.152888,-1.404647,-1.091896,-0.783203,-0.689500,0.923828,0.206055,...,-1.550188,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.334248,-0.507271,-0.508977
100006,0.0,-0.577531,-0.307114,-0.711432,0.177857,-0.653403,-0.928711,-0.680105,-0.280518,-1.375000,...,-1.550188,1.693287,-0.138614,1.613777,0.0,1.613777,0.0,-0.271209,-0.523054,-0.471789
100007,0.0,-0.577531,-0.431181,-0.213740,-0.361753,-0.068746,0.562988,-0.892525,-0.279785,0.191284,...,-1.550188,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.077109,-0.317542,-0.420660


In [30]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a,set2a
0,100002,1.0,-0.577531,0.313221,-0.478099,-0.166152,-0.507239,-0.149780,1.506891,0.747070,...,0.645083,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,5.053162,0.986709,0.860543
1,100003,0.0,-0.577531,0.933556,1.725433,0.592656,1.599963,-1.252930,-0.166811,0.511719,...,0.645083,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.839862,-0.548398,-0.659948
2,100004,0.0,-0.577531,-0.927449,-1.152888,-1.404647,-1.091896,-0.783203,-0.689500,0.923828,...,-1.550188,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.334248,-0.507271,-0.508977
3,100006,0.0,-0.577531,-0.307114,-0.711432,0.177857,-0.653403,-0.928711,-0.680105,-0.280518,...,-1.550188,1.693287,-0.138614,1.613777,0.0,1.613777,0.0,-0.271209,-0.523054,-0.471789
4,100007,0.0,-0.577531,-0.431181,-0.213740,-0.361753,-0.068746,0.562988,-0.892525,-0.279785,...,-1.550188,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.077109,-0.317542,-0.420660


In [31]:
### indexを削除
df_train = df_train.drop(columns=['index'])
df_test = df_test.drop(columns=['index'])
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a,set2a
0,100001,-0.577531,-0.307114,-0.075104,-0.451791,-0.239271,-0.146484,-0.734184,0.024017,-0.051636,...,0.645083,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.369428,-0.423673,-0.522810
1,100005,-0.577531,-0.637959,-0.934826,-0.671920,-0.970093,1.079102,-0.464476,-0.891113,-1.172852,...,0.645083,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,0.969472,0.423197,0.166750
2,100013,-0.577531,0.313221,0.159594,2.943903,0.247943,-0.127563,-0.916815,-0.886230,0.797852,...,-1.550188,1.693287,-0.138614,1.613777,0.0,1.613777,0.0,-0.452737,-0.475807,-0.396008
3,100028,2.192099,1.347112,2.424818,1.511669,2.805819,0.399658,0.472284,0.221558,0.847656,...,0.645083,1.693287,-0.138614,1.613777,0.0,1.613777,0.0,-0.305554,-0.503358,-0.400113
4,100038,0.807284,0.106443,0.065768,0.342100,0.235763,-0.783203,0.686768,0.082153,0.279785,...,-1.550188,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,1.168560,1.199636,1.036076


In [32]:
### infをnanへ置換
df_train = df_train.replace([np.inf, -np.inf], np.nan)
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a,set2a
0,100002,1.0,-0.577531,0.313221,-0.478099,-0.166152,-0.507239,-0.149780,1.506891,0.747070,...,0.645083,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,5.053162,0.986709,0.860543
1,100003,0.0,-0.577531,0.933556,1.725433,0.592656,1.599963,-1.252930,-0.166811,0.511719,...,0.645083,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.839862,-0.548398,-0.659948
2,100004,0.0,-0.577531,-0.927449,-1.152888,-1.404647,-1.091896,-0.783203,-0.689500,0.923828,...,-1.550188,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.334248,-0.507271,-0.508977
3,100006,0.0,-0.577531,-0.307114,-0.711432,0.177857,-0.653403,-0.928711,-0.680105,-0.280518,...,-1.550188,1.693287,-0.138614,1.613777,0.0,1.613777,0.0,-0.271209,-0.523054,-0.471789
4,100007,0.0,-0.577531,-0.431181,-0.213740,-0.361753,-0.068746,0.562988,-0.892525,-0.279785,...,-1.550188,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.077109,-0.317542,-0.420660


In [33]:
### infをnanへ置換
df_test = df_test.replace([np.inf, -np.inf], np.nan)
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a,set2a
0,100001,-0.577531,-0.307114,-0.075104,-0.451791,-0.239271,-0.146484,-0.734184,0.024017,-0.051636,...,0.645083,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.369428,-0.423673,-0.522810
1,100005,-0.577531,-0.637959,-0.934826,-0.671920,-0.970093,1.079102,-0.464476,-0.891113,-1.172852,...,0.645083,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,0.969472,0.423197,0.166750
2,100013,-0.577531,0.313221,0.159594,2.943903,0.247943,-0.127563,-0.916815,-0.886230,0.797852,...,-1.550188,1.693287,-0.138614,1.613777,0.0,1.613777,0.0,-0.452737,-0.475807,-0.396008
3,100028,2.192099,1.347112,2.424818,1.511669,2.805819,0.399658,0.472284,0.221558,0.847656,...,0.645083,1.693287,-0.138614,1.613777,0.0,1.613777,0.0,-0.305554,-0.503358,-0.400113
4,100038,0.807284,0.106443,0.065768,0.342100,0.235763,-0.783203,0.686768,0.082153,0.279785,...,-1.550188,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,1.168560,1.199636,1.036076


In [34]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 694.44 MB
Memory usage of optimization is 691273616.00 MB
Decreased by -99543771.2%
Memory usage of dataframe is 109.99 MB
Memory usage of optimization is 109381664.00 MB
Decreased by -99450647.5%


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a,set2a
0,100002,1.0,-0.577637,0.313232,-0.478027,-0.166138,-0.507324,-0.149780,1.506836,0.747070,...,0.645020,-0.590332,-0.138672,-0.619629,0.0,-0.619629,0.0,5.054688,0.986816,0.860352
1,100003,0.0,-0.577637,0.933594,1.725586,0.592773,1.599609,-1.252930,-0.166870,0.511719,...,0.645020,-0.590332,-0.138672,-0.619629,0.0,-0.619629,0.0,-0.839844,-0.548340,-0.660156
2,100004,0.0,-0.577637,-0.927246,-1.153320,-1.404297,-1.091797,-0.783203,-0.689453,0.923828,...,-1.549805,-0.590332,-0.138672,-0.619629,0.0,-0.619629,0.0,-0.334229,-0.507324,-0.508789
3,100006,0.0,-0.577637,-0.307129,-0.711426,0.177856,-0.653320,-0.928711,-0.680176,-0.280518,...,-1.549805,1.693359,-0.138672,1.614258,0.0,1.614258,0.0,-0.271240,-0.522949,-0.471680
4,100007,0.0,-0.577637,-0.431152,-0.213745,-0.361816,-0.068726,0.562988,-0.892578,-0.279785,...,-1.549805,-0.590332,-0.138672,-0.619629,0.0,-0.619629,0.0,-0.077087,-0.317627,-0.420654


### modelを生成

In [35]:
# trainデータセット作成
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [36]:
# testデータセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [37]:
### training keras
# 1層目のモデル
# pred_train_1a, pred_train_1bは、学習データのクロスバリデーションでの予測値
# pred_test_1a, pred_test_1bは、テストデータの予測値
model_1b = Model1Lgb()
pred_train_1b, pred_test_1b = predict_cv(model_1b, x_train, y_train, x_test)

[LightGBM] [Info] Number of positive: 18633, number of negative: 211996
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.051154 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 120272
[LightGBM] [Info] Number of data points in the train set: 230629, number of used features: 555
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080792 -> initscore=-2.431633
[LightGBM] [Info] Start training from score -2.431633
[1]	train's auc: 0.644903	valid's auc: 0.642063
[2]	train's auc: 0.67987	valid's auc: 0.674709
[3]	train's auc: 0.717547	valid's auc: 0.707722
[4]	train's auc: 0.739199	valid's auc: 0.729453
[5]	train's auc: 0.743623	valid's auc: 0.734607
[6]	train's auc: 0.74752	valid's auc: 0.737768
[7]	train's auc: 0.772769	valid's auc: 0.765197
[8]	train's auc: 0.771276	valid's auc: 0.763679
[9]	train's auc: 0.773992	valid's auc: 0.766186
[10]

[160]	train's auc: 0.797028	valid's auc: 0.786688
[161]	train's auc: 0.797125	valid's auc: 0.786734
[162]	train's auc: 0.79718	valid's auc: 0.786766
[163]	train's auc: 0.797288	valid's auc: 0.786828
[164]	train's auc: 0.797268	valid's auc: 0.786763
[165]	train's auc: 0.797316	valid's auc: 0.786781
[166]	train's auc: 0.797425	valid's auc: 0.786868
[167]	train's auc: 0.797502	valid's auc: 0.786903
[168]	train's auc: 0.797545	valid's auc: 0.786914
[169]	train's auc: 0.797595	valid's auc: 0.786937
[170]	train's auc: 0.797635	valid's auc: 0.78691
[171]	train's auc: 0.797726	valid's auc: 0.786973
[172]	train's auc: 0.797738	valid's auc: 0.786946
[173]	train's auc: 0.797814	valid's auc: 0.786993
[174]	train's auc: 0.797836	valid's auc: 0.78696
[175]	train's auc: 0.797902	valid's auc: 0.787003
[176]	train's auc: 0.797987	valid's auc: 0.78706
[177]	train's auc: 0.798056	valid's auc: 0.787103
[178]	train's auc: 0.798062	valid's auc: 0.787071
[179]	train's auc: 0.798082	valid's auc: 0.787072
[180

[325]	train's auc: 0.807244	valid's auc: 0.7919
[326]	train's auc: 0.807297	valid's auc: 0.791916
[327]	train's auc: 0.807343	valid's auc: 0.791923
[328]	train's auc: 0.807387	valid's auc: 0.791965
[329]	train's auc: 0.807439	valid's auc: 0.79197
[330]	train's auc: 0.807487	valid's auc: 0.791984
[331]	train's auc: 0.807561	valid's auc: 0.792033
[332]	train's auc: 0.807607	valid's auc: 0.792085
[333]	train's auc: 0.807646	valid's auc: 0.792095
[334]	train's auc: 0.807703	valid's auc: 0.792116
[335]	train's auc: 0.807776	valid's auc: 0.792178
[336]	train's auc: 0.807828	valid's auc: 0.792189
[337]	train's auc: 0.807853	valid's auc: 0.79221
[338]	train's auc: 0.8079	valid's auc: 0.792221
[339]	train's auc: 0.807943	valid's auc: 0.792214
[340]	train's auc: 0.807982	valid's auc: 0.792206
[341]	train's auc: 0.80804	valid's auc: 0.792253
[342]	train's auc: 0.808082	valid's auc: 0.792269
[343]	train's auc: 0.808144	valid's auc: 0.792288
[344]	train's auc: 0.808189	valid's auc: 0.792328
[345]	t

[493]	train's auc: 0.814922	valid's auc: 0.793604
[494]	train's auc: 0.814959	valid's auc: 0.79361
[495]	train's auc: 0.814981	valid's auc: 0.793612
[496]	train's auc: 0.815028	valid's auc: 0.793626
[497]	train's auc: 0.81507	valid's auc: 0.793634
[498]	train's auc: 0.815114	valid's auc: 0.793637
[499]	train's auc: 0.815161	valid's auc: 0.793641
[500]	train's auc: 0.815207	valid's auc: 0.793651
[501]	train's auc: 0.81526	valid's auc: 0.793658
[502]	train's auc: 0.815306	valid's auc: 0.793673
[503]	train's auc: 0.815345	valid's auc: 0.793672
[504]	train's auc: 0.815376	valid's auc: 0.793672
[505]	train's auc: 0.815414	valid's auc: 0.793668
[506]	train's auc: 0.815468	valid's auc: 0.79369
[507]	train's auc: 0.815512	valid's auc: 0.793688
[508]	train's auc: 0.815563	valid's auc: 0.793708
[509]	train's auc: 0.815614	valid's auc: 0.793729
[510]	train's auc: 0.815654	valid's auc: 0.793735
[511]	train's auc: 0.815695	valid's auc: 0.793746
[512]	train's auc: 0.815741	valid's auc: 0.793747
[513

[661]	train's auc: 0.821771	valid's auc: 0.794694
[662]	train's auc: 0.821807	valid's auc: 0.79469
[663]	train's auc: 0.821851	valid's auc: 0.79469
[664]	train's auc: 0.821889	valid's auc: 0.794689
[665]	train's auc: 0.821924	valid's auc: 0.794699
[666]	train's auc: 0.821964	valid's auc: 0.794698
[667]	train's auc: 0.821988	valid's auc: 0.794698
[668]	train's auc: 0.822028	valid's auc: 0.794707
[669]	train's auc: 0.82207	valid's auc: 0.794698
[670]	train's auc: 0.822112	valid's auc: 0.794695
[671]	train's auc: 0.822156	valid's auc: 0.794697
[672]	train's auc: 0.822201	valid's auc: 0.794703
[673]	train's auc: 0.822222	valid's auc: 0.79472
[674]	train's auc: 0.822257	valid's auc: 0.794727
[675]	train's auc: 0.822297	valid's auc: 0.794741
[676]	train's auc: 0.822337	valid's auc: 0.79475
[677]	train's auc: 0.822368	valid's auc: 0.794752
[678]	train's auc: 0.822413	valid's auc: 0.794762
[679]	train's auc: 0.822453	valid's auc: 0.794763
[680]	train's auc: 0.822499	valid's auc: 0.794766
[681]

...

[1235]	train's auc: 0.840473	valid's auc: 0.79302
[1236]	train's auc: 0.84049	valid's auc: 0.79302
[1237]	train's auc: 0.840523	valid's auc: 0.793016
[1238]	train's auc: 0.840549	valid's auc: 0.793005
[1239]	train's auc: 0.840563	valid's auc: 0.79301
[1240]	train's auc: 0.84058	valid's auc: 0.793011
[1241]	train's auc: 0.840607	valid's auc: 0.793014
[1242]	train's auc: 0.840637	valid's auc: 0.793016
[1243]	train's auc: 0.84065	valid's auc: 0.793016
[1244]	train's auc: 0.840683	valid's auc: 0.793007
[1245]	train's auc: 0.840714	valid's auc: 0.793006
[1246]	train's auc: 0.840744	valid's auc: 0.792999
[1247]	train's auc: 0.840761	valid's auc: 0.79301
[1248]	train's auc: 0.840782	valid's auc: 0.793011
[1249]	train's auc: 0.840815	valid's auc: 0.793026
[1250]	train's auc: 0.840848	valid's auc: 0.793031
[1251]	train's auc: 0.840881	valid's auc: 0.793028
[1252]	train's auc: 0.840908	valid's auc: 0.793028
[1253]	train's auc: 0.84094	valid's auc: 0.793022
[1254]	train's auc: 0.840974	valid's au

[1400]	train's auc: 0.84501	valid's auc: 0.792905
[1401]	train's auc: 0.845037	valid's auc: 0.792905
[1402]	train's auc: 0.845064	valid's auc: 0.792905
[1403]	train's auc: 0.845098	valid's auc: 0.792899
[1404]	train's auc: 0.845118	valid's auc: 0.792893
[1405]	train's auc: 0.845147	valid's auc: 0.792889
[1406]	train's auc: 0.845175	valid's auc: 0.792888
[1407]	train's auc: 0.845201	valid's auc: 0.792879
[1408]	train's auc: 0.84522	valid's auc: 0.792876
[1409]	train's auc: 0.845247	valid's auc: 0.792881
[1410]	train's auc: 0.845271	valid's auc: 0.792883
[1411]	train's auc: 0.845293	valid's auc: 0.792882
[1412]	train's auc: 0.845323	valid's auc: 0.792885
[1413]	train's auc: 0.845342	valid's auc: 0.792886
[1414]	train's auc: 0.845368	valid's auc: 0.792887
[1415]	train's auc: 0.84539	valid's auc: 0.792884
[1416]	train's auc: 0.84542	valid's auc: 0.792884
[1417]	train's auc: 0.845447	valid's auc: 0.792888
[1418]	train's auc: 0.845474	valid's auc: 0.792884
[1419]	train's auc: 0.845486	valid'

[1562]	train's auc: 0.84926	valid's auc: 0.792809
[1563]	train's auc: 0.849288	valid's auc: 0.792806
[1564]	train's auc: 0.849308	valid's auc: 0.792806
[1565]	train's auc: 0.849332	valid's auc: 0.792808
[1566]	train's auc: 0.849359	valid's auc: 0.792806
[1567]	train's auc: 0.849376	valid's auc: 0.792803
[1568]	train's auc: 0.849404	valid's auc: 0.79281
[1569]	train's auc: 0.849431	valid's auc: 0.792815
[1570]	train's auc: 0.849455	valid's auc: 0.792808
[1571]	train's auc: 0.849476	valid's auc: 0.792812
[1572]	train's auc: 0.849499	valid's auc: 0.792815
[1573]	train's auc: 0.849528	valid's auc: 0.792819
[1574]	train's auc: 0.849548	valid's auc: 0.792817
[1575]	train's auc: 0.849568	valid's auc: 0.792815
[1576]	train's auc: 0.849586	valid's auc: 0.792811
[1577]	train's auc: 0.849602	valid's auc: 0.792814
[1578]	train's auc: 0.849622	valid's auc: 0.792816
[1579]	train's auc: 0.849648	valid's auc: 0.792818
[1580]	train's auc: 0.849676	valid's auc: 0.792821
[1581]	train's auc: 0.849696	vali

[1724]	train's auc: 0.853153	valid's auc: 0.792815
[1725]	train's auc: 0.853183	valid's auc: 0.792811
[1726]	train's auc: 0.853208	valid's auc: 0.792813
[1727]	train's auc: 0.853228	valid's auc: 0.79281
[1728]	train's auc: 0.853249	valid's auc: 0.792803
[1729]	train's auc: 0.85327	valid's auc: 0.792805
[1730]	train's auc: 0.853287	valid's auc: 0.792809
[1731]	train's auc: 0.853309	valid's auc: 0.792811
[1732]	train's auc: 0.853336	valid's auc: 0.792818
[1733]	train's auc: 0.853359	valid's auc: 0.792815
[1734]	train's auc: 0.853381	valid's auc: 0.792813
[1735]	train's auc: 0.853407	valid's auc: 0.792815
[1736]	train's auc: 0.853427	valid's auc: 0.792815
[1737]	train's auc: 0.853448	valid's auc: 0.792815
[1738]	train's auc: 0.853473	valid's auc: 0.792813
[1739]	train's auc: 0.853498	valid's auc: 0.792816
[1740]	train's auc: 0.853525	valid's auc: 0.792814
[1741]	train's auc: 0.853547	valid's auc: 0.792806
[1742]	train's auc: 0.853567	valid's auc: 0.792802
[1743]	train's auc: 0.853584	vali

[1886]	train's auc: 0.856767	valid's auc: 0.792655
[1887]	train's auc: 0.856781	valid's auc: 0.79266
[1888]	train's auc: 0.856795	valid's auc: 0.792661
[1889]	train's auc: 0.856817	valid's auc: 0.792653
[1890]	train's auc: 0.856845	valid's auc: 0.792656
[1891]	train's auc: 0.856871	valid's auc: 0.792652
[1892]	train's auc: 0.856891	valid's auc: 0.792652
[1893]	train's auc: 0.85691	valid's auc: 0.792656
[1894]	train's auc: 0.856926	valid's auc: 0.792655
[1895]	train's auc: 0.856956	valid's auc: 0.792641
[1896]	train's auc: 0.856976	valid's auc: 0.792639
[1897]	train's auc: 0.856997	valid's auc: 0.792642
[1898]	train's auc: 0.857021	valid's auc: 0.792639
[1899]	train's auc: 0.857045	valid's auc: 0.792633
[1900]	train's auc: 0.857071	valid's auc: 0.792628
[1901]	train's auc: 0.857084	valid's auc: 0.792628
[1902]	train's auc: 0.857099	valid's auc: 0.792628
[1903]	train's auc: 0.857119	valid's auc: 0.792625
[1904]	train's auc: 0.857145	valid's auc: 0.792616
[1905]	train's auc: 0.857163	vali

In [38]:
pred_test_1b

array([0.03820515, 0.10308749, 0.05807468, ..., 0.00881218, 0.01743904,
       0.2005258 ])

In [39]:
pred_train_1b

array([0.25429376, 0.02202631, 0.03599927, ..., 0.03249072, 0.09429715,
       0.0845682 ])

In [40]:
pred_1b = []
for a in np.array(pred_train_1b):
    pred_1b.append(a)

#pred_1b = np.array(pred_1b)
len(pred_1b)

307506

In [41]:
print(type(pred_1b))

<class 'list'>


In [42]:
### 予測値を特徴量としてデータフレームを作成
sk_id_curr = df_train['SK_ID_CURR'].copy().to_list()
train_x_2 = pd.DataFrame({'SK_ID_CURR': sk_id_curr, 'set3a': pred_1b})
print(train_x_2.shape)
train_x_2.head()

(307506, 2)


,SK_ID_CURR,set3a
0,100002,0.254294
1,100003,0.022026
2,100004,0.035999
3,100006,0.021632
4,100007,0.025333


In [43]:
train_x_2.to_csv(
    path_or_buf="./home-credit-default-risk/exports/train_set3a.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

In [44]:
pred_1b = []
for a in np.array(pred_test_1b):
    pred_1b.append(a)

#pred_1b = np.array(pred_1b)
len(pred_1b)

48744

In [45]:
### 予測値を特徴量としてデータフレームを作成
sk_id_curr = df_test['SK_ID_CURR'].copy().to_list()
test_x_2 = pd.DataFrame({'SK_ID_CURR': sk_id_curr, 'set3a': pred_1b})
print(test_x_2.shape)
test_x_2.head()

(48744, 2)


,SK_ID_CURR,set3a
0,100001,0.038205
1,100005,0.103087
2,100013,0.058075
3,100028,0.035603
4,100038,0.184927


In [46]:
test_x_2.to_csv(
    path_or_buf="./home-credit-default-risk/exports/test_set3a.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

In [47]:
del df
del x_train
del y_train
del x_test
del df_train
del df_test
del model_1b
gc.collect()

2

## 特徴量生成2